## Look Twice, Learn Better : How SimCLR transformed Computer Vision

``` Rishita Agarwal ```

## Motivation behind this project topic?

The growth of digital images has created a havoc in machine learning, we have a lot of visual data, yet supervised learning approaches act as a bottleneck due to the lack of high quality labeled data. This is one of the major challenges in computer vision today.

One of the solution which can be proposed for this can be manual annotation of the data, but manual labelling is expensive, time consuming and many times impractical at large scale. Some times it may also require expert knowledge in a particular domain.

**Unsupervised learning** offers a more promising path by leveraging the large amount of unlabeled data available. Though, it is not that easy to produce results as good as supervised learning. Some of the key requirements are -
1. The learned representation should be **generalizable** to diverse downstream tasks as well.
2. The approach should **scale computationally** with larger datasets and model sizes.
3. Features should capture **meaningful semantic** information.

<div style="display: flex; justify-content: center;">
    <img src="https://www.edushots.com/upload/articles-images/b35a6ab4259fcd2fa572cc62333ac5ec15371617.jpg" alt="SimCLR Framework" width="45%" style="margin-right: 10px;"/>
    <img src="https://media.geeksforgeeks.org/wp-content/uploads/20231213175718/Self-660.png" alt="SimCLR Results" width="45%"/>
</div>
<p style="text-align: center;"><em>Figure: Unsupervised Learning (left) and Self Supervised Learning (right)</em></p>

SimCLR addressed these requirements in a simplistic manner, still managing to achieve state-of-the-art results, grabing my attention to this topic. The idea was simple yet innovative, which actually was derived from the essential components of existing methods.


## History and current works?

The initial methods relating to **self supervised visual representations** led to this exploration. Let me talk more about these methods.

It all started with **handcrafted pretext** tasks like predicting image rotation or colorizing grayscale images. These methods were a good start to the finding a good representation, but it was noticed that the learnt representations were more specific to the pretext task rather than being general purpose.

Then, approaches like **InstDisc and CPC** introduced the concept of contrastive objectives (Fig. given below) but still relied on complex architectures or memory banks to store representations.

<div style="display: flex; justify-content: center;">
    <img src="https://insights.willogy.io/assets/static/contrastive_learning_intuition.42db587.208b1cf168018c6226966d0407c62134.jpg" alt="Contrastive Learning Concept" width="50%"/>
</div>
<p style="text-align: center;"><em>Figure: Visualization of contrastive learning approach</em></p>

Finally **SimCLR** simplied the approach of contrastive learning dramatically, showing that with right data augmentation, loss function and projection head, superior results with a straightforward framework could be achieved.
Even after SimCLR more methods based on its insights were built, like, MoCo v2, SimSiam etc. to further improve the self supervision, trying to reduce the need for negative examples. CLIP and CLAP are 2 multimodal approaches to Contrastive learning, applying CL to image-text and audio-text pairs. I will give a basic introduction to these methods towards the end of this blog.

## Diving deep into Contrastive Learning and SimCLR's approach

**Contrastive learning** is an approach that learns representations by comparing similar and dissimilar samples. The fundamental idea behind this concept is: Similar items ("Positive") should be closer together and dissimilar items ("Negative") should be further apart.

It is a self supervised technique which learns meaningful representation without any explicitly labeled data.

**SimCLR (Simple Framework for Contrastive Learning of Visual Representation) Framework** 

This framework consists of 4 major components - 
1. Data Augmentation Module - In this module, each input image goes through stochastic augmentation twice, to generate 2 different views.

``` 
X -> [augmentation] -> x̃ᵢ
X -> [augmentation] -> x̃ⱼ
```

The augmentation sequence is as follows :
- **Random Cropping** - After cropping the image randomly, it should again be resized to original size.
- **Random color distortion** - Include color dropping, brightness, contrast etc.
- **Random Gaussian blur** - (write more on this)

The most important combination of transformation is that of cropping (spatial transformation) and color distortion (appearance transformation). A good reason for this is, that without color distortion the networks can exploit the shortcut of matching color histograms, rather than actually learning semantic features.

<div style="display: flex; justify-content: center;">
    <img src="https://www.researchgate.net/publication/362707124/figure/fig2/AS:11431281079262702@1660628166563/Proposed-transformations.ppm" alt="Data Augmentation" width="45%" style="margin-right: 10px;"/>
    <img src="https://img-blog.csdnimg.cn/20201013134203801.png?,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3h6MTMwODU3OTM0MA==,size_16,color_FFFFFF,t_70#pic_center" alt="Data Augmentation" width="45%" style="margin-right: 10px;"/>
</div>
<p style="text-align: center;"><em>Figure: Data Augmentation Example (left) and Composition of Augmentation Techniques (right)</em></p>


2. Base Encoder Network

SimCLR employs a standard CNN (ResNet architecture) as its base encoder. Given an augmented image x̃, the encoder generates a representation vector -

``` h = f(x̃) = ResNet(x̃) ```

where h ∈ ℝᵈ is the output after the average pooling layer.

3. Projection Head
This head maps the representation h to a space z, where contrastive loss is applied -

``` z = g(h) = W₂σ(W₁h) ```

Where σ is a ReLU nonlinearity, and W₁ and W₂ are learnable weights. 
In this research paper, they have made a relatively simple MLP with one hidden layer as the projection head. 

## Contrastive Loss and Where should it be applied?

It was noticed that applying the contrastive loss directly on the representations h hurts their quality for downstream tasks. The projection head allows the model to discard information that may be useful for the downstream task but not for the contrastive task.

Crucially, after training is complete, the projection head is discarded, and the **representation h is used for downstream tasks.**

#### Contrastive Loss function 
A special form of contrastive loss called NT-Xent (Normalized Temperature scaled Cross Entropy Loss) is used. For a positive pair (i, j) :

``` ℓᵢ,ⱼ = -log(exp(sim(zᵢ,zⱼ)/τ) / Σₖ₌₁²ᴺ 1[k≠i]exp(sim(zᵢ,zₖ)/τ)) ```

Where:

- sim(u,v) = u·v/‖u‖‖v‖ is the cosine similarity
- τ is a temperature parameter that controls the concentration level of the distribution
- The sum is over all 2N examples in the batch (including the other augmented views)

This loss effectively treats each augmented image in the batch as a single positive example (the other augmented view of the same image) and 2N-2 negative examples (all other augmented images)

<div style="display: flex; justify-content: center;">
    <img src="https://sthalles.github.io/assets/contrastive-self-supervised/cover.png" alt="SimCLR Framework" width="50%"/>
</div>
<p style="text-align: center;"><em>Figure: SimCLR Framework</em></p>
 

## Implementation in Pytorch

## CLIP and CLAP